In [ ]:
PREPROCESSED_PATH = "E:\Projects\Story-Telling-Using-Show-Attend-and-tell"

def preProBuildWordVocab(sentence_iterator, word_count_threshold=5):
    # borrowed this function from NeuralTalk
    print ('preprocessing word counts and creating vocab based on word count threshold %d' % (word_count_threshold, ))

    word_counts = {}
    nsents = 0

    for sent in sentence_iterator:
        nsents += 1
        tmp_sent = sent.lower().split(' ')
        if '' in tmp_sent:
            tmp_sent.remove('')

        for w in tmp_sent:
           word_counts[w] = word_counts.get(w, 0) + 1

    vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
    print ('filtered words from %d to %d' % (len(word_counts), len(vocab)))

    ixtoword = {}
    ixtoword[0] = '<bos>'
    ixtoword[1] = '<eos>'
    ixtoword[2] = '<pad>'
    ixtoword[3] = '<unk>'

    wordtoix = {}
    wordtoix['<bos>'] = 0
    wordtoix['<eos>'] = 1
    wordtoix['<pad>'] = 2
    wordtoix['<unk>'] = 3

    for idx, w in enumerate(vocab):
        wordtoix[w] = idx + 4
        ixtoword[idx+4] = w

    word_counts['<eos>'] = nsents
    word_counts['<bos>'] = nsents
    word_counts['<pad>'] = nsents
    word_counts['<unk>'] = nsents

    bias_init_vector = np.array([1.0 * word_counts[ ixtoword[i] ] for i in ixtoword])
    bias_init_vector /= np.sum(bias_init_vector) # normalize to frequencies
    bias_init_vector = np.log(bias_init_vector)
    bias_init_vector -= np.max(bias_init_vector) # shift to nice numeric range

    return wordtoix, ixtoword, bias_init_vector


In [ ]:
import pickle
import numpy as np
batch_size = 50 # Being support batch_size
num_boxes = 50 # number of Detected regions in each image
feats_dim = 4096 # feature dimensions of each regions
project_dim = 1024 # project the features to one vector, which is 1024 dimensions

sentRNN_lstm_dim = 512 # the sentence LSTM hidden units
sentRNN_FC_dim = 1024 # the fully connected units
wordRNN_lstm_dim = 512 # the word LSTM hidden units
word_embed_dim = 1024 # the learned embedding vectors for the words

S_max = 6
N_max = 30
T_stop = 0.5

n_epochs = 500
learning_rate = 0.0001
name = "nota" 

  
img2paragraph = pickle.load(open('img2paragraph'  ,'rb'))
all_sentences = []
for key, paragraph in img2paragraph.items():
    for each_sent in paragraph[1]:
        each_sent.replace(',', ' ,')
        all_sentences.append(each_sent)
word2idx, idx2word, bias_init_vector = preProBuildWordVocab(all_sentences, word_count_threshold=2)
np.save('idx2word_batch', idx2word)

img2paragraph_modify = {}
for img_name, img_paragraph in img2paragraph.items():
    img_paragraph_1 = img_paragraph[1]

    if '' in img_paragraph_1:
        img_paragraph_1.remove('')
    if ' ' in paragraph[1]:
        img_paragraph_1.remove(' ')   
    img_num_sents = len(img_paragraph_1)
    if img_num_sents > S_max:
        img_num_sents = S_max
    img_num_distribution = np.zeros([S_max], dtype=np.int32)
    img_num_distribution[img_num_sents-1:] = 1

    img_captions_matrix = np.ones([S_max, N_max+1], dtype=np.int32) * 2 # zeros([6, 50])
    for idx, img_sent in enumerate(img_paragraph_1):
        # the number of sentences is img_num_sents
        if idx == img_num_sents:
            break

        # because we treat the ',' as a word
        img_sent = img_sent.replace(',', ' ,')

       
        if len(img_sent)>=3 and img_sent[0] == ' ' and img_sent[1] != ' ':
            img_sent = img_sent[1:]
        elif len(img_sent)>=3 and img_sent[0] == ' ' and img_sent[1] == ' ' and img_sent[2] != ' ':
            img_sent = img_sent[2:]

        # Be careful the last part in a sentence, like this:
        # '...world.'
        # '...world. '
        if len(img_sent)>=1 and img_sent[-1] == '.':
            img_sent = img_sent[0:-1]
        elif len(img_sent)>=2 and img_sent[-1] == ' ' and img_sent[-2] == '.':
            img_sent = img_sent[0:-2]

        # Last, we add the <bos> and the <eos> in each sentences
        img_sent = '<bos> ' + img_sent + ' <eos>'

        for idy, word in enumerate(img_sent.lower().split(' ')):
            # because the biggest number of words in a sentence is N_max, here is 50
            if idy == N_max:
                break

            if word in word2idx:
                img_captions_matrix[idx, idy] = word2idx[word]
            else:
                img_captions_matrix[idx, idy] = word2idx['<unk>']

    img2paragraph_modify[str(img_name)] = [img_num_distribution, img_captions_matrix]
with open('img2paragraph_modify_batch', 'wb') as f:
    pickle.dump(img2paragraph_modify, f)

In [ ]:
class RegionPooling_HierarchicalRNN():
    def __init__(self, n_words,
                       batch_size,
                       num_boxes,
                       feats_dim,
                       project_dim,
                       sentRNN_lstm_dim,
                       sentRNN_FC_dim,
                       wordRNN_lstm_dim,
                       S_max,
                       N_max,
                       word_embed_dim,
                       bias_init_vector=None):

        self.n_words = n_words
        self.batch_size = batch_size
        self.num_boxes = num_boxes # 50
        self.feats_dim = feats_dim # 4096
        self.project_dim = project_dim # 1024
        self.S_max = S_max # 6
        self.N_max = N_max # 50
        self.word_embed_dim = word_embed_dim # 1024

        self.sentRNN_lstm_dim = sentRNN_lstm_dim # 512 hidden size
        self.sentRNN_FC_dim = sentRNN_FC_dim # 1024 in fully connected layer
        self.wordRNN_lstm_dim = wordRNN_lstm_dim # 512 hidden size

        # word embedding, parameters of embedding
        # embedding shape: n_words x wordRNN_lstm_dim
        with tf.device('/gpu:0'):
            self.Wemb = tf.Variable(tf.random_uniform([n_words, word_embed_dim], -0.1, 0.1), name='Wemb')
        #self.bemb = tf.Variable(tf.zeros([word_embed_dim]), name='bemb')

        # regionPooling_W shape: 4096 x 1024
        # regionPooling_b shape: 1024
        self.regionPooling_W = tf.Variable(tf.random_uniform([feats_dim, project_dim], -0.1, 0.1), name='regionPooling_W')
        self.regionPooling_b = tf.Variable(tf.zeros([project_dim]), name='regionPooling_b')

        # sentence LSTM
        self.sent_LSTM = tf.nn.rnn_cell.BasicLSTMCell(sentRNN_lstm_dim, state_is_tuple=True)

        # logistic classifier
        self.logistic_Theta_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, 2], -0.1, 0.1), name='logistic_Theta_W')
        self.logistic_Theta_b = tf.Variable(tf.zeros(2), name='logistic_Theta_b')

        # fc1_W: 512 x 1024, fc1_b: 1024
        # fc2_W: 1024 x 1024, fc2_b: 1024
        self.fc1_W = tf.Variable(tf.random_uniform([sentRNN_lstm_dim, sentRNN_FC_dim], -0.1, 0.1), name='fc1_W')
        self.fc1_b = tf.Variable(tf.zeros(sentRNN_FC_dim), name='fc1_b')
        self.fc2_W = tf.Variable(tf.random_uniform([sentRNN_FC_dim, 1024], -0.1, 0.1), name='fc2_W')
        self.fc2_b = tf.Variable(tf.zeros(1024), name='fc2_b')

        # word LSTM
        self.word_LSTM = tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim, state_is_tuple=True)
        self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([self.word_LSTM,self.word_LSTM], state_is_tuple=True)
        self.word_LSTM = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(wordRNN_lstm_dim,state_is_tuple=True) for _ in range(2)])

        self.embed_word_W = tf.Variable(tf.random_uniform([wordRNN_lstm_dim, n_words], -0.1,0.1), name='embed_word_W')
        if bias_init_vector is not None:
            self.embed_word_b = tf.Variable(bias_init_vector.astype(np.float32), name='embed_word_b')
            self.embed_word_b = tf.Variable(tf.zeros([n_words]), name='embed_word_b')
   
    def generate_model(self):
        # feats: 1 x 50 x 4096
        feats = tf.placeholder(tf.float32, [1, self.num_boxes, self.feats_dim])
        # tmp_feats: 50 x 4096
        tmp_feats = tf.reshape(feats, [-1, self.feats_dim])

        # project_vec_all: 50 x 4096 * 4096 x 1024 + 1024 --> 50 x 1024
        project_vec_all = tf.matmul(tmp_feats, self.regionPooling_W) + self.regionPooling_b
        project_vec_all = tf.reshape(project_vec_all, [1, 50, self.project_dim])
        project_vec = tf.reduce_max(project_vec_all, reduction_indices=1)

        # initialize the sent_LSTM state
        sent_state = self.sent_LSTM.zero_state(batch_size=1, dtype=tf.float32)

        # save the generated paragraph to list, here I named generated_sents
        generated_paragraph = []

        # pred
        pred_re = []

        T_stop = tf.constant(0.5)

        print ('Start build the generation model: ')
        
        for i in range(0, self.S_max):
            if i > 0:
                tf.get_variable_scope().reuse_variables()

            with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
                sent_output, sent_state = self.sent_LSTM(project_vec, sent_state)

            
            with tf.name_scope('fc1'):
                hidden1 = tf.nn.relu( tf.matmul(sent_output, self.fc1_W) + self.fc1_b )
            with tf.name_scope('fc2'):
                sent_topic_vec = tf.nn.relu( tf.matmul(hidden1, self.fc2_W) + self.fc2_b )

            sentRNN_logistic_mu = tf.nn.xw_plus_b(sent_output, self.logistic_Theta_W, self.logistic_Theta_b)
            pred = tf.nn.softmax(sentRNN_logistic_mu)
            pred_re.append(pred)

            # save the generated sentence to list, named generated_sent
            generated_sent = []

            topic = tf.nn.rnn_cell.LSTMStateTuple(sent_topic_vec[:, 0:512], sent_topic_vec[:, 512:])
            word_state = (topic, topic)
            # word RNN, unrolled to N_max time steps
            for j in range(0, self.N_max):
                if j > 0:
                    tf.get_variable_scope().reuse_variables()

                if j == 0:
                    with tf.device('/gpu:0'):
                        # get word embedding of BOS (index = 0)
                          current_embed = tf.nn.embedding_lookup(self.Wemb, tf.zeros([1], dtype=tf.int64))

                with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):

                    word_output, word_state = self.word_LSTM(current_embed, word_state)

            
                logit_words = tf.nn.xw_plus_b(word_output, self.embed_word_W, self.embed_word_b)
                max_prob_index = tf.argmax(logit_words, 1)[0]
                generated_sent.append(max_prob_index)

                with tf.device('/gpu:0'):
                    current_embed = tf.nn.embedding_lookup(self.Wemb, max_prob_index)
                    current_embed = tf.expand_dims(current_embed, 0)

            generated_paragraph.append(generated_sent)

        # return feats, generated_paragraph, pred_re, --cxp
        return feats, generated_paragraph, pred_re, generated_sent


    

In [ ]:
import pandas as pd
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
import time
import h5py
import os
import tensorflow as tf
import time
import random
import matplotlib.pyplot as plt
import numpy as np
def test():
    candidates = {}
    start_time = time.time()
    model_path = './model-380'

    idx2word = pd.Series(np.load('idx2word_batch.npy').tolist())

    test_feats_path = 'im2p_train_output1.h5'
    test_output_file = h5py.File(test_feats_path, 'r')
    test_feats = test_output_file.get('feats')

    test_imgs_full_path_lists = open('train_images.txt').read().splitlines()
    test_imgs_names = list(map(lambda x: os.path.basename(x).split('.')[0], test_imgs_full_path_lists))
    from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
    print_tensors_in_checkpoint_file(file_name='./model-0', tensor_name='', all_tensors=False)


    with tf.variable_scope(tf.get_variable_scope()) as scope:
        test_model = RegionPooling_HierarchicalRNN(n_words = len(word2idx),
                                                   batch_size = batch_size,
                                                   num_boxes = num_boxes,
                                                   feats_dim = feats_dim,
                                                   project_dim = project_dim,
                                                   sentRNN_lstm_dim = sentRNN_lstm_dim,
                                                   sentRNN_FC_dim = sentRNN_FC_dim,
                                                   wordRNN_lstm_dim = wordRNN_lstm_dim,
                                                   S_max = S_max,
                                                   N_max = N_max,
                                                   word_embed_dim = word_embed_dim,
                                                   bias_init_vector = bias_init_vector)

        tf_feats, tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors = test_model.generate_model()

    with tf.Session() as sess:
        print ("start creating session tree")
 
        saver = tf.train.Saver()
        saver.restore(sess, model_path)
        for var in tf.global_variables():
          print('all variables: ' + var.op.name)
        for var in tf.trainable_variables():
          print('normal variable: ' + var.op.name)

        img2idx = {}
        for idx, img in enumerate(test_imgs_names):
            img2idx[img] = idx

        test_fd = open('HRNN_results.txt', 'w')
        for idx, img_name in enumerate(test_imgs_names):
            if idx % 100 == 0:
                print (idx, img_name)
            test_fd.write(img_name + '\n')

            each_paragraph = []
            current_paragraph = ""

            current_feats_index = img2idx[img_name]
            current_feats = test_feats[current_feats_index]
            current_feats = np.reshape(current_feats, [1, 50, 4096])

            generated_paragraph_indexes, pred, sent_topic_vectors = sess.run(
                                                                             [tf_generated_paragraph, tf_pred_re, tf_sent_topic_vectors],
                                                                             feed_dict={
                                                                                 tf_feats: current_feats
                                                                             })

            for sent_index in generated_paragraph_indexes:
                each_sent = []
                for word_index in sent_index:
                    each_sent.append(idx2word[word_index])
                each_paragraph.append(each_sent)

            for idx, each_sent in enumerate(each_paragraph):
              
                if pred[idx][0][0] <= T_stop:
                    break
                current_sent = ''
                for each_word in each_sent:
                    current_sent += each_word + ' '
                current_sent = current_sent.replace('<eos> ', '')
                current_sent = current_sent.replace('<pad> ', '')
                current_sent = current_sent + '.'
                current_sent = current_sent.replace(' .', '.')
                current_sent = current_sent.replace(' ,', ',')
                current_paragraph +=current_sent
                if idx != len(each_paragraph) - 1:
                    current_paragraph += ' '


            test_fd.write(current_paragraph + '\n')
        test_fd.close()
        print ("Time cost: " + str(time.time()-start_time) + 'seconds')


In [ ]:
test()

In [ ]:
Wemb (DT_FLOAT) [9937,1024]
Wemb_1 (DT_FLOAT) [9937,1024]
Wemb_1/Adam (DT_FLOAT) [9937,1024]
Wemb_1/Adam_1 (DT_FLOAT) [9937,1024]
Wemb_2 (DT_FLOAT) [9937,1024]
Wemb_2/Adam (DT_FLOAT) [9937,1024]
Wemb_2/Adam_1 (DT_FLOAT) [9937,1024]
Wemb_3 (DT_FLOAT) [9937,1024]
Wemb_3/Adam (DT_FLOAT) [9937,1024]
Wemb_3/Adam_1 (DT_FLOAT) [9937,1024]
basic_lstm_cell/bias (DT_FLOAT) [2048]
basic_lstm_cell/bias/Adam (DT_FLOAT) [2048]
basic_lstm_cell/bias/Adam_1 (DT_FLOAT) [2048]
basic_lstm_cell/kernel (DT_FLOAT) [1536,2048]
basic_lstm_cell/kernel/Adam (DT_FLOAT) [1536,2048]
basic_lstm_cell/kernel/Adam_1 (DT_FLOAT) [1536,2048]
beta1_power (DT_FLOAT) []
beta1_power_1 (DT_FLOAT) []
beta1_power_2 (DT_FLOAT) []
beta2_power (DT_FLOAT) []
beta2_power_1 (DT_FLOAT) []
beta2_power_2 (DT_FLOAT) []
embed_word_W (DT_FLOAT) [512,9937]
embed_word_W_1 (DT_FLOAT) [512,9937]
embed_word_W_1/Adam (DT_FLOAT) [512,9937]
embed_word_W_1/Adam_1 (DT_FLOAT) [512,9937]
embed_word_W_2 (DT_FLOAT) [512,9937]
embed_word_W_2/Adam (DT_FLOAT) [512,9937]
embed_word_W_2/Adam_1 (DT_FLOAT) [512,9937]
embed_word_W_3 (DT_FLOAT) [512,9937]
embed_word_W_3/Adam (DT_FLOAT) [512,9937]
embed_word_W_3/Adam_1 (DT_FLOAT) [512,9937]
embed_word_b (DT_FLOAT) [9937]
embed_word_b_1 (DT_FLOAT) [9937]
embed_word_b_2 (DT_FLOAT) [9937]
embed_word_b_3 (DT_FLOAT) [9937]
embed_word_b_3/Adam (DT_FLOAT) [9937]
embed_word_b_3/Adam_1 (DT_FLOAT) [9937]
embed_word_b_4 (DT_FLOAT) [9937]
embed_word_b_5 (DT_FLOAT) [9937]
embed_word_b_5/Adam (DT_FLOAT) [9937]
embed_word_b_5/Adam_1 (DT_FLOAT) [9937]
embed_word_b_6 (DT_FLOAT) [9937]
embed_word_b_7 (DT_FLOAT) [9937]
embed_word_b_7/Adam (DT_FLOAT) [9937]
embed_word_b_7/Adam_1 (DT_FLOAT) [9937]
fc1_W (DT_FLOAT) [512,1024]
fc1_W_1 (DT_FLOAT) [512,1024]
fc1_W_1/Adam (DT_FLOAT) [512,1024]
fc1_W_1/Adam_1 (DT_FLOAT) [512,1024]
fc1_W_2 (DT_FLOAT) [512,1024]
fc1_W_2/Adam (DT_FLOAT) [512,1024]
fc1_W_2/Adam_1 (DT_FLOAT) [512,1024]
fc1_W_3 (DT_FLOAT) [512,1024]
fc1_W_3/Adam (DT_FLOAT) [512,1024]
fc1_W_3/Adam_1 (DT_FLOAT) [512,1024]
fc1_b (DT_FLOAT) [1024]
fc1_b_1 (DT_FLOAT) [1024]
fc1_b_1/Adam (DT_FLOAT) [1024]
fc1_b_1/Adam_1 (DT_FLOAT) [1024]
fc1_b_2 (DT_FLOAT) [1024]
fc1_b_2/Adam (DT_FLOAT) [1024]
fc1_b_2/Adam_1 (DT_FLOAT) [1024]
fc1_b_3 (DT_FLOAT) [1024]
fc1_b_3/Adam (DT_FLOAT) [1024]
fc1_b_3/Adam_1 (DT_FLOAT) [1024]
fc2_W (DT_FLOAT) [1024,1024]
fc2_W_1 (DT_FLOAT) [1024,1024]
fc2_W_1/Adam (DT_FLOAT) [1024,1024]
fc2_W_1/Adam_1 (DT_FLOAT) [1024,1024]
fc2_W_2 (DT_FLOAT) [1024,1024]
fc2_W_2/Adam (DT_FLOAT) [1024,1024]
fc2_W_2/Adam_1 (DT_FLOAT) [1024,1024]
fc2_W_3 (DT_FLOAT) [1024,1024]
fc2_W_3/Adam (DT_FLOAT) [1024,1024]
fc2_W_3/Adam_1 (DT_FLOAT) [1024,1024]
fc2_b (DT_FLOAT) [1024]
fc2_b_1 (DT_FLOAT) [1024]
fc2_b_1/Adam (DT_FLOAT) [1024]
fc2_b_1/Adam_1 (DT_FLOAT) [1024]
fc2_b_2 (DT_FLOAT) [1024]
fc2_b_2/Adam (DT_FLOAT) [1024]
fc2_b_2/Adam_1 (DT_FLOAT) [1024]
fc2_b_3 (DT_FLOAT) [1024]
fc2_b_3/Adam (DT_FLOAT) [1024]
fc2_b_3/Adam_1 (DT_FLOAT) [1024]
logistic_Theta_W (DT_FLOAT) [512,2]
logistic_Theta_W_1 (DT_FLOAT) [512,2]
logistic_Theta_W_1/Adam (DT_FLOAT) [512,2]
logistic_Theta_W_1/Adam_1 (DT_FLOAT) [512,2]
logistic_Theta_W_2 (DT_FLOAT) [512,2]
logistic_Theta_W_2/Adam (DT_FLOAT) [512,2]
logistic_Theta_W_2/Adam_1 (DT_FLOAT) [512,2]
logistic_Theta_W_3 (DT_FLOAT) [512,2]
logistic_Theta_W_3/Adam (DT_FLOAT) [512,2]
logistic_Theta_W_3/Adam_1 (DT_FLOAT) [512,2]
logistic_Theta_b (DT_FLOAT) [2]
logistic_Theta_b_1 (DT_FLOAT) [2]
logistic_Theta_b_1/Adam (DT_FLOAT) [2]
logistic_Theta_b_1/Adam_1 (DT_FLOAT) [2]
logistic_Theta_b_2 (DT_FLOAT) [2]
logistic_Theta_b_2/Adam (DT_FLOAT) [2]
logistic_Theta_b_2/Adam_1 (DT_FLOAT) [2]
logistic_Theta_b_3 (DT_FLOAT) [2]
logistic_Theta_b_3/Adam (DT_FLOAT) [2]
logistic_Theta_b_3/Adam_1 (DT_FLOAT) [2]
multi_rnn_cell/cell_0/basic_lstm_cell/bias (DT_FLOAT) [2048]
multi_rnn_cell/cell_0/basic_lstm_cell/bias/Adam (DT_FLOAT) [2048]
multi_rnn_cell/cell_0/basic_lstm_cell/bias/Adam_1 (DT_FLOAT) [2048]
multi_rnn_cell/cell_0/basic_lstm_cell/kernel (DT_FLOAT) [1536,2048]
multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Adam (DT_FLOAT) [1536,2048]
multi_rnn_cell/cell_0/basic_lstm_cell/kernel/Adam_1 (DT_FLOAT) [1536,2048]
multi_rnn_cell/cell_1/basic_lstm_cell/bias (DT_FLOAT) [2048]
multi_rnn_cell/cell_1/basic_lstm_cell/bias/Adam (DT_FLOAT) [2048]
multi_rnn_cell/cell_1/basic_lstm_cell/bias/Adam_1 (DT_FLOAT) [2048]
multi_rnn_cell/cell_1/basic_lstm_cell/kernel (DT_FLOAT) [1024,2048]
multi_rnn_cell/cell_1/basic_lstm_cell/kernel/Adam (DT_FLOAT) [1024,2048]
multi_rnn_cell/cell_1/basic_lstm_cell/kernel/Adam_1 (DT_FLOAT) [1024,2048]
regionPooling_W (DT_FLOAT) [4096,1024]
regionPooling_W_1 (DT_FLOAT) [4096,1024]
regionPooling_W_1/Adam (DT_FLOAT) [4096,1024]
regionPooling_W_1/Adam_1 (DT_FLOAT) [4096,1024]
regionPooling_W_2 (DT_FLOAT) [4096,1024]
regionPooling_W_2/Adam (DT_FLOAT) [4096,1024]
regionPooling_W_2/Adam_1 (DT_FLOAT) [4096,1024]
regionPooling_W_3 (DT_FLOAT) [4096,1024]
regionPooling_W_3/Adam (DT_FLOAT) [4096,1024]
regionPooling_W_3/Adam_1 (DT_FLOAT) [4096,1024]
regionPooling_b (DT_FLOAT) [1024]
regionPooling_b_1 (DT_FLOAT) [1024]
regionPooling_b_1/Adam (DT_FLOAT) [1024]
regionPooling_b_1/Adam_1 (DT_FLOAT) [1024]
regionPooling_b_2 (DT_FLOAT) [1024]
regionPooling_b_2/Adam (DT_FLOAT) [1024]
regionPooling_b_2/Adam_1 (DT_FLOAT) [1024]
regionPooling_b_3 (DT_FLOAT) [1024]
regionPooling_b_3/Adam (DT_FLOAT) [1024]
regionPooling_b_3/Adam_1 (DT_FLOAT) [1024]